In [1]:
import talib

In [2]:
api_key = open('/tmp/.secret/alphavantage_api_key.txt').read()

In [3]:
symbol = "RTX"

In [4]:
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}"

In [5]:
import requests
res = requests.get(url)
print(res.status_code)

200


In [6]:
raw_data:dict = res.json()

In [7]:
#Convert alphavantage time series to pandas dataframe
import pandas as pd
res_df = pd.DataFrame(raw_data["Time Series (Daily)"],dtype=float).T

In [8]:
#convert panda dataframe index to datetime
res_df.index = pd.to_datetime(res_df.index)
#sort the dataframe by index ascending
res_df.sort_index(ascending=True, inplace=True)
#calculate talib SMA from pandas dataframe
res_df["SMA20"] = talib.SMA(res_df["4. close"].astype(float), timeperiod=20).astype(float)
res_df["SMA50"] = talib.SMA(res_df["4. close"].astype(float), timeperiod=50).astype(float)
#calculate talib RSI from pandas dataframe
res_df["RSI"] = talib.RSI(res_df["4. close"].astype(float), timeperiod=14).astype(float)
res_df.dtypes

1. open                 float64
2. high                 float64
3. low                  float64
4. close                float64
5. adjusted close       float64
6. volume               float64
7. dividend amount      float64
8. split coefficient    float64
SMA20                   float64
SMA50                   float64
RSI                     float64
dtype: object

In [9]:
res_df.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,SMA20,SMA50,RSI
2023-02-27,99.98,100.030,99.02,99.16,99.16,2986716.0,0.0,1.0,99.3745,99.1884,48.461006
2023-02-28,99.27,99.472,97.82,98.09,98.09,4469983.0,0.0,1.0,99.3435,99.1512,43.494369
2023-03-01,98.04,98.570,97.50,98.04,98.04,3740060.0,0.0,1.0,99.2530,99.1422,43.271197
2023-03-02,97.50,99.325,97.50,99.00,99.00,4134579.0,0.0,1.0,99.2820,99.1554,48.712515
2023-03-03,99.18,99.320,98.26,99.02,99.02,5072818.0,0.0,1.0,99.3850,99.1850,48.822649


In [12]:
#PLotly graph with candlestick and sma
#Make plotly subplot of primary and secondary y axis
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=2, cols=1,
                        specs=[[{"secondary_y": True}],
                             [{"secondary_y": True}]],
                        shared_xaxes=True,
                        row_heights=[10, 8])

# #Plot the close price
# fig = px.line(res_df, x=res_df.index, y=["4. close",  "SMA10", "SMA200"])

#Plot the candlestick
fig.add_trace(
    go.Candlestick(
        x=res_df.index,
        open=res_df["1. open"],
        high=res_df["2. high"],
        low=res_df["3. low"],
        close=res_df["4. close"],
        name="Candlestick"
    ),row=1,col=1
)

#Add one more SMA20 plot into row=1, col=1 subplot
fig.add_trace(
    go.Scatter(
        x=res_df.index,
        y=res_df["SMA20"],
        name="SMA20",
        line=dict(color="red")
    ),row=1,col=1
)

#Add one more SMA50 plot into row=1, col=1 subplot with color blue
fig.add_trace(
    go.Scatter(
        x=res_df.index,
        y=res_df["SMA50"],
        name="SMA50",
        line=dict(color="blue")
    ),row=1,col=1
)

#subplot RSI in a separate graph
fig.add_scatter(x=res_df.index, y=res_df["RSI"], row=2, col=1
                 ,name="RSI")



# fig.update_yaxes(range=[ 80,110]) 
# print(fig.layout)
fig.update_layout(
        margin=dict(l=20, r=20, t=20, b=20),
        paper_bgcolor="LightSteelBlue",
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis= {'rangeslider': {'visible': False},
              'showticklabels': False},
        xaxis2= {'rangeslider': {'visible': True}},
        yaxis = dict(title="candles", range=[80,110]),
        yaxis3 = dict(title="RSI", range=[0,80])
)

# fig.update_layout(width=1600, height=400)
# fig.update_xaxes(rangeslider_visible=True)
#fig.update_yaxes(range=[0,100])
print(fig.layout)
fig.show()



Layout({
    'legend': {'orientation': 'h', 'x': 1, 'xanchor': 'right', 'y': 1.02, 'yanchor': 'bottom'},
    'margin': {'b': 20, 'l': 20, 'r': 20, 't': 20},
    'paper_bgcolor': 'LightSteelBlue',
    'showlegend': True,
    'template': '...',
    'xaxis': {'anchor': 'y',
              'domain': [0.0, 0.94],
              'matches': 'x2',
              'rangeslider': {'visible': False},
              'showticklabels': False},
    'xaxis2': {'anchor': 'y3', 'domain': [0.0, 0.94], 'rangeslider': {'visible': True}},
    'yaxis': {'anchor': 'x', 'domain': [0.5277777777777778, 1.0], 'range': [80, 110], 'title': {'text': 'candles'}},
    'yaxis2': {'anchor': 'x', 'overlaying': 'y', 'side': 'right'},
    'yaxis3': {'anchor': 'x2', 'domain': [0.0, 0.37777777777777777], 'range': [0, 80], 'title': {'text': 'RSI'}},
    'yaxis4': {'anchor': 'x2', 'overlaying': 'y3', 'side': 'right'}
})


In [11]:
print(fig.layout)

Layout({
    'legend': {'orientation': 'h', 'x': 1, 'xanchor': 'right', 'y': 1.02, 'yanchor': 'bottom'},
    'margin': {'b': 20, 'l': 20, 'r': 20, 't': 20},
    'paper_bgcolor': 'LightSteelBlue',
    'showlegend': True,
    'template': '...',
    'xaxis': {'anchor': 'y',
              'domain': [0.0, 0.94],
              'matches': 'x2',
              'rangeslider': {'visible': False},
              'showticklabels': False},
    'xaxis2': {'anchor': 'y3', 'domain': [0.0, 0.94], 'rangeslider': {'visible': True}},
    'yaxis': {'anchor': 'x', 'domain': [0.5277777777777778, 1.0], 'range': [80, 110], 'title': {'text': 'candles'}},
    'yaxis2': {'anchor': 'x', 'overlaying': 'y', 'side': 'right'},
    'yaxis3': {'anchor': 'x2', 'domain': [0.0, 0.37777777777777777], 'range': [0, 80], 'title': {'text': 'RSI'}},
    'yaxis4': {'anchor': 'x2', 'overlaying': 'y3', 'side': 'right'}
})
